In [ ]:
#检测两份数据是否可以相乘

In [ ]:
import rasterio
import numpy as np

# 定义文件路径
tif_file_1 = 'M:/2015-2020data/0jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km_revise/re_ssp126_2021.tif'  # 热指数文件
tif_file_2 = 'F:\人口数据\lanscan_origin\landscan-global-2021-assets\landscan-global-2021.tif'  # 人口数据文件

# 检查热指数数据 (TIFF文件)
with rasterio.open(tif_file_1) as src1:
    tif1_bounds = src1.bounds
    tif1_transform = src1.transform
    print("热指数数据边界:", tif1_bounds)
    print("热指数数据变换矩阵:", tif1_transform)

    # 检查CRS是否匹配
    if src1.crs.to_string() == 'EPSG:4326':
        print("热指数数据CRS: WGS 84")
    else:
        print("热指数数据CRS:", src1.crs)

# 检查人口数据 (TIFF文件)
with rasterio.open(tif_file_2) as src2:
    tif2_bounds = src2.bounds
    tif2_transform = src2.transform
    print("人口数据边界:", tif2_bounds)
    print("人口数据变换矩阵:", tif2_transform)

    # 检查CRS是否匹配
    if src2.crs.to_string() == 'EPSG:4326':
        print("人口数据CRS: WGS 84")
    else:
        print("人口数据CRS:", src2.crs)

# 对比两个数据集以判断是否可以相乘
# 这仅仅是一个简单的对比，可能需要更详细的分析
if tif1_bounds.left >= tif2_bounds.left and tif1_bounds.right <= tif2_bounds.right and \
   tif1_bounds.bottom >= tif2_bounds.bottom and tif1_bounds.top <= tif2_bounds.top:
    print("数据集在空间上是兼容的，可以相乘。")
else:
    print("数据集在空间上不兼容，无法直接相乘。")

In [ ]:
#根据人口数据调整热指数数据空间extent

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os

# 定义新路径
population_tif = 'F:/人口数据/lanscan_origin/landscan-global-2021-assets/landscan-global-2021.tif'
heat_index_tif = 'M:/2015-2020data/0jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km/re_ssp126_2021.tif'
output_tif = 'M:/2015-2020data/0jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km_revise/re_ssp126_2021.tif'

# 创建输出目录（如果不存在）
output_dir = os.path.dirname(output_tif)
os.makedirs(output_dir, exist_ok=True)

# 获取人口数据的目标元数据
with rasterio.open(population_tif) as src_pop:
    dst_crs = src_pop.crs
    dst_transform = src_pop.transform
    dst_width = src_pop.width
    dst_height = src_pop.height

# 打开热指数数据并进行重投影和重采样
with rasterio.open(heat_index_tif) as src:
    # 计算新的变换矩阵和大小
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    
    # 更新元数据
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': dst_transform,
        'width': dst_width,
        'height': dst_height
    })
    
    # 创建输出文件并重采样
    with rasterio.open(output_tif, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=dst_transform,
                dst_crs=dst_crs,
                resampling=Resampling.bilinear
            )

print(f"新热指数数据已处理并保存为: {output_tif}")


In [ ]:
#根据热指数数据调整人口数据空间extent，有效的

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os

# 定义新路径
heat_index_tif = 'M:/2015-2020data/0jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km/re_ssp126_2021.tif'
population_tif = 'F:/人口数据/lanscan_origin/landscan-global-2021-assets/landscan-global-2021.tif'
output_tif = 'F:/人口数据/lanscan_origin/landscan-global-2021-assets/landscan-global-2021_360.tif'

# 获取热指数数据的目标元数据
with rasterio.open(heat_index_tif) as src_heat:
    dst_crs = src_heat.crs
    dst_transform = src_heat.transform
    dst_width = src_heat.width
    dst_height = src_heat.height

# 打开人口数据并调整范围
with rasterio.open(population_tif) as src_pop:
    # 更新元数据
    kwargs = src_pop.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': dst_transform,
        'width': dst_width,
        'height': dst_height
    })
    
    # 创建新的文件并调整范围
    with rasterio.open(output_tif, 'w', **kwargs) as dst:
        for i in range(1, src_pop.count + 1):
            reproject(
                source=rasterio.band(src_pop, i),
                destination=rasterio.band(dst, i),
                src_transform=src_pop.transform,
                src_crs=src_pop.crs,
                dst_transform=dst_transform,
                dst_crs=dst_crs,
                resampling=Resampling.bilinear
            )

print(f"人口数据已调整空间范围并保存为: {output_tif}")

In [ ]:
#根据热指数修改人口数据

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os

# 定义基础路径
input_base_dir = 'M:/2015-2020data'
output_base_dir = 'M:/2015-2020data'
population_tif = 'F:/人口数据/lanscan_origin/landscan-global-2021-assets/landscan-global-2021.tif'
heat_index_tif = 'M:/2015-2020data/Heat index2021-0.125_revise/Heat index_AgERA5_20210101.tif'

# 获取热指数数据的元数据（坐标参考系、范围、大小等）
with rasterio.open(heat_index_tif) as src_heat:
    src_crs = src_heat.crs
    src_transform = src_heat.transform
    src_width = src_heat.width
    src_height = src_heat.height
    src_bounds = src_heat.bounds  # 获取热指数数据的空间范围

# 读取人口数据的元数据
with rasterio.open(population_tif) as src_pop:
    dst_crs = src_pop.crs
    dst_transform = src_pop.transform
    dst_width = src_pop.width
    dst_height = src_pop.height
    dst_bounds = src_pop.bounds  # 获取人口数据的空间范围

# 计算新的投影和分辨率，使其与热指数数据的空间范围匹配
transform, width, height = calculate_default_transform(
    dst_crs, src_crs, dst_width, dst_height, *src_bounds)

# 创建输出文件路径
output_file = 'F:/人口数据/lanscan_origin/landscan-global-2021-assets/landscan-global-2021_revised.tif'

# 使用热指数数据的空间范围重采样和重投影人口数据
kwargs = src_pop.meta.copy()
kwargs.update({
    'crs': src_crs,  # 更新为热指数数据的CRS
    'transform': transform,
    'width': width,
    'height': height
})

# 打开输出文件进行写操作
with rasterio.open(output_file, 'w', **kwargs) as dst:
    # 重新打开人口数据集进行重投影
    with rasterio.open(population_tif) as src_pop:
        for i in range(1, src_pop.count + 1):
            reproject(
                source=rasterio.band(src_pop, i),
                destination=rasterio.band(dst, i),
                src_transform=src_pop.transform,
                src_crs=src_pop.crs,
                dst_transform=transform,
                dst_crs=src_crs,
                resampling=Resampling.nearest)


In [ ]:
#将原来的改成left=0.0, bottom=-60.0, right=360.0, top=90.0

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os

# 定义基础路径
input_base_dir = 'M:/2015-2020data'
output_base_dir = 'M:/2015-2020data'
population_tif = 'F:/人口数据/change/popdenREScou_RE_0.125_revise/pppden_2014_1km_Aggregated_REScou_RE_revise.tif'

# 获取人口数据的目标元数据
with rasterio.open(population_tif) as src_pop:
    dst_crs = src_pop.crs
    dst_transform = src_pop.transform
    dst_width = src_pop.width
    dst_height = src_pop.height

# 定义日期范围
years = range(2014, 2021)
months = range(1, 13)
days = range(1, 32)

# 处理每个日期的热指数文件
for year in years:
    input_dir = os.path.join(input_base_dir, f'Heat index{year}-0.125')
    output_dir = os.path.join(output_base_dir, f'Heat index{year}-0.125_revise')
    
    # 创建输出目录（如果不存在）
    os.makedirs(output_dir, exist_ok=True)
    
    for month in months:
        for day in days:
            # 生成文件名
            date_str = f'{year}{month:02d}{day:02d}'
            input_file = os.path.join(input_dir, f'Heat index_AgERA5_{date_str}.tif')
            output_file = os.path.join(output_dir, f'Heat index_AgERA5_{date_str}.tif')
            
            # 检查文件是否存在
            if not os.path.exists(input_file):
                continue

            # 打开热指数数据并进行重投影和重采样
            with rasterio.open(input_file) as src:
                transform, width, height = calculate_default_transform(
                    src.crs, dst_crs, src.width, src.height, *src.bounds)
                
                kwargs = src.meta.copy()
                kwargs.update({
                    'crs': dst_crs,
                    'transform': dst_transform,
                    'width': dst_width,
                    'height': dst_height
                })
                
                with rasterio.open(output_file, 'w', **kwargs) as dst:
                    for i in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, i),
                            destination=rasterio.band(dst, i),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=dst_transform,
                            dst_crs=dst_crs,
                            resampling=Resampling.bilinear
                        )
            
            print(f'处理完成: {output_file}')

print("所有热指数数据处理完成。")

In [ ]:
#之前处理的是2000-2014 现在是2014-2020
#之前处理的是.nc数据，现在处理的是.tif数据，历史数据
#之前处理的是0.25°*0.25°的数据，现在需要处理的是0.125°*0.125°数据
#修改！计算气温数据和人口数据乘积时 不用固定阈值，而用新阈值 hi_yuzhi_day_r1i1p1f1_gn_2000_canshu0.2.nc
#用worldpop数据相比之前的数据，需要额外注意nodata不参与计算，nodata区域的最后结果仍然是nodata

In [ ]:
import rasterio
import numpy as np
import os
import datetime

def is_leap_year(year):
    """判断是否是闰年"""
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def process_data(years):
    for year in years:
        start_date = datetime.date(year, 1, 1)
        end_date = datetime.date(year, 12, 31)
        delta = datetime.timedelta(days=1)

        date = start_date
        while date <= end_date:
            date_str = date.strftime('%Y%m%d')  # 日期格式为YYYYMMDD
            date_md_str = date.strftime('%m%d')  # 日期格式为MMDD

            weather_file = f'M:/2015-2020data/Heat index{year}-0.125_revise/Heat index_AgERA5_{date_str}.tif'
            output_file = f'M:/2015-2020data/jisuan18.3_0.1_pop_0.125_tif_change/ssp126_rasters/re_ssp126_{date_str}.tif'

            if date.month == 2 and date.day == 29:
                threshold_file = f'L:/CMIP6-1yuzhi0.125/yuzhi_canshu0.1/yuzhi_canshu_0.1_0228.tif'
            else:
                threshold_file = f'L:/CMIP6-1yuzhi0.125/yuzhi_canshu0.1/yuzhi_canshu_0.1_{date_md_str}.tif'

            # 检查输入文件是否存在
            if not os.path.exists(weather_file):
                print("File not found: {}".format(weather_file))
                date += delta
                continue

            if not os.path.exists(threshold_file):
                print("File not found: {}".format(threshold_file))
                date += delta
                continue

            with rasterio.open(weather_file) as weather_src, rasterio.open(threshold_file) as threshold_src:
                weather_data = weather_src.read(1)
                threshold_data = threshold_src.read(1)
                weather_transform = weather_src.transform
                weather_nodata = weather_src.nodata

            # 计算阈值调整后的数据
            adjusted_data = np.maximum(weather_data - threshold_data, 0)

            # 结果数据去除人口影响
            result_data = np.where((weather_data == weather_nodata) | (threshold_data == threshold_src.nodata),
                                   weather_nodata,
                                   adjusted_data)

            # 确保输出目录存在
            output_dir = os.path.dirname(output_file)
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            with rasterio.open(
                output_file,
                'w',
                driver='GTiff',
                height=weather_src.height,
                width=weather_src.width,
                count=1,
                dtype=weather_data.dtype,
                crs=weather_src.crs,
                transform=weather_transform,
                nodata=weather_nodata
            ) as dst:
                dst.write(result_data, 1)

            print("Processed and saved:", output_file)
            date += delta

# 处理2000-2010年的数据
years = list(range(2000, 2020))
process_data(years)

In [ ]:
#通过.tif计算年度和

In [ ]:
import rasterio
import numpy as np
import os
import datetime

def is_leap_year(year):
    """判断是否是闰年"""
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def calculate_annual_sum(scenario, start_year, end_year):
    for year in range(start_year, end_year + 1):
        # 定义输入和输出路径
        input_dir = f'M:/2015-2020data/jisuan18.3_0.1_pop_0.125_tif_change/{scenario}_rasters/'
        output_dir = f'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/{scenario}_rasters_origin/'
        output_file = f'{output_dir}re_{scenario}_{year}.tif'

        # 确保输出目录存在
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # 初始化累加器
        annual_sum = None
        nodata_value = None
        transform = None
        crs = None

        # 获取一年中的所有日期
        start_date = datetime.date(year, 1, 1)
        end_date = datetime.date(year + 1, 1, 1) if is_leap_year(year) else datetime.date(year + 1, 1, 1)
        delta = datetime.timedelta(days=1)

        date = start_date
        while date < end_date:
            date_str = date.strftime('%Y%m%d')
            input_file = os.path.join(input_dir, f're_{scenario}_{date_str}.tif')

            try:
                # 读取栅格数据
                with rasterio.open(input_file) as src:
                    data = src.read(1).astype(np.float32)
                    nodata_value = src.nodata if nodata_value is None else nodata_value
                    transform = src.transform if transform is None else transform
                    crs = src.crs if crs is None else crs

                    # 初始化累加器
                    if annual_sum is None:
                        annual_sum = np.zeros_like(data, dtype=np.float32)
                        nan_mask = np.zeros_like(data, dtype=bool)

                    # 处理nodata区域
                    mask = (data == nodata_value) | np.isnan(data)
                    nan_mask = nan_mask | mask
                    data[mask] = 0

                    # 累加数据
                    annual_sum += data

            except rasterio.errors.RasterioIOError:
                # 跳过不存在的文件
                pass

            date += delta

        if annual_sum is not None:
            # 恢复nan和nodata区域
            annual_sum[nan_mask] = np.nan

            # 保存年度总和数据到新的TIFF文件
            with rasterio.open(
                output_file,
                'w',
                driver='GTiff',
                height=annual_sum.shape[0],
                width=annual_sum.shape[1],
                count=1,
                dtype=np.float32,
                crs=crs,
                transform=transform,
                nodata=nodata_value
            ) as dst:
                dst.write(annual_sum, 1)
                print(f"Processed and saved: {output_file}")
        else:
            print(f"No data processed for year {year}")

# 定义情景和年份
scenario = 'ssp126'
start_year = 2000
end_year = 2001

# 计算每年所有日期之和并保存
calculate_annual_sum(scenario, start_year, end_year)

In [ ]:
#对0.25°*0.25°原始数据进行降尺度处理，转变为0.0083333333°*0.0083333333°数据，
#原始数据的extent 为top为90，left为0，right为360，bottom为-60

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os
import numpy as np

# 定义输入和输出文件路径
input_file = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_origin/re_ssp126_2000.tif'
output_file = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km/re_ssp126_2000.tif'

# 确保输出目录存在
output_folder = os.path.dirname(output_file)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 打开输入文件
with rasterio.open(input_file) as src:
    # 计算新分辨率
    new_transform, width, height = calculate_default_transform(
        src.crs, src.crs, src.width, src.height, 
        *src.bounds, dst_width=int(src.width * (0.125 / 0.0083333333)), dst_height=int(src.height * (0.125 / 0.0083333333))
    )

    # 设置输出的元数据
    profile = src.profile
    profile.update({
        'driver': 'GTiff',
        'height': height,
        'width': width,
        'transform': new_transform,
        'crs': src.crs,
        'dtype': rasterio.float32  # 根据需要更改数据类型
    })

    # 重采样并保存到输出文件
    with rasterio.open(output_file, 'w', **profile) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=new_transform,
                dst_crs=src.crs,
                resampling=Resampling.nearest  # 使用最近邻插值方法，可以根据需要更改
            )

    print(f"文件已重采样并保存到：{output_file}")

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os
import numpy as np

# 定义年份范围
years = range(2000, 2021)

# 定义输入和输出文件模板
input_file_template = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_origin/re_ssp126_{year}.tif'
output_file_template = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km/re_ssp126_{year}.tif'

for year in years:
    # 格式化文件路径
    input_file = input_file_template.format(year=year)
    output_file = output_file_template.format(year=year)

    # 确保输出目录存在
    output_folder = os.path.dirname(output_file)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 打开输入文件
    with rasterio.open(input_file) as src:
        # 计算新分辨率
        new_transform, width, height = calculate_default_transform(
            src.crs, src.crs, src.width, src.height,
            *src.bounds, dst_width=int(src.width * (0.125 / 0.0083333333)), dst_height=int(src.height * (0.125 / 0.0083333333))
        )

        # 设置输出的元数据
        profile = src.profile
        profile.update({
            'driver': 'GTiff',
            'height': height,
            'width': width,
            'transform': new_transform,
            'crs': src.crs,
            'dtype': rasterio.float32  # 根据需要更改数据类型
        })

        # 重采样并保存到输出文件
        with rasterio.open(output_file, 'w', **profile) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=new_transform,
                    dst_crs=src.crs,
                    resampling=Resampling.nearest  # 使用最近邻插值方法，可以根据需要更改
                )

        print(f"文件已重采样并保存到：{output_file}")

In [ ]:
#检查气温数据和人口数据空间是否匹配
#结果是
#气温数据维度: OrderedDict([('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 360), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1440), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 600)])
#气温数据经度范围: 0.125 359.875
#气温数据纬度范围: -59.875 89.875
#人口数据边界: BoundingBox(left=0.0, bottom=-60.0, right=360.0, top=90.0)
#人口数据变换矩阵: | 0.25, 0.00, 0.00|
#| 0.00,-0.27, 90.00|
#| 0.00, 0.00, 1.00|
#人口数据CRS: WGS 84
#数据集在空间上是兼容的，可以相乘。

In [ ]:
import rasterio
import numpy as np

# 定义文件路径
tif_file_1 = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km/re_ssp126_2000.tif'  # 热指数文件
tif_file_2 = 'F:\人口数据\origin\ppp_2000_1km_Aggregated.tif'  # 人口数据文件

# 检查热指数数据 (TIFF文件)
with rasterio.open(tif_file_1) as src1:
    tif1_bounds = src1.bounds
    tif1_transform = src1.transform
    print("热指数数据边界:", tif1_bounds)
    print("热指数数据变换矩阵:", tif1_transform)

    # 检查CRS是否匹配
    if src1.crs.to_string() == 'EPSG:4326':
        print("热指数数据CRS: WGS 84")
    else:
        print("热指数数据CRS:", src1.crs)

# 检查人口数据 (TIFF文件)
with rasterio.open(tif_file_2) as src2:
    tif2_bounds = src2.bounds
    tif2_transform = src2.transform
    print("人口数据边界:", tif2_bounds)
    print("人口数据变换矩阵:", tif2_transform)

    # 检查CRS是否匹配
    if src2.crs.to_string() == 'EPSG:4326':
        print("人口数据CRS: WGS 84")
    else:
        print("人口数据CRS:", src2.crs)

# 对比两个数据集以判断是否可以相乘
# 这仅仅是一个简单的对比，可能需要更详细的分析
if tif1_bounds.left >= tif2_bounds.left and tif1_bounds.right <= tif2_bounds.right and \
   tif1_bounds.bottom >= tif2_bounds.bottom and tif1_bounds.top <= tif2_bounds.top:
    print("数据集在空间上是兼容的，可以相乘。")
else:
    print("数据集在空间上不兼容，无法直接相乘。")

In [ ]:
#气候数据和人口数据无法直接相乘，需要根据人口数据调整气候数据。

In [ ]:
import rasterio
from rasterio.warp import reproject, Resampling
import os
import numpy as np

# 定义年份范围
years = range(2001, 2021)

# 定义输入和输出文件路径模板
tif_file_1_template = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km/re_ssp126_{year}.tif'  # 热指数文件模板
tif_file_2_template = 'F:/人口数据/origin/ppp_{year}_1km_Aggregated.tif'  # 原始人口数据文件模板
output_file_template = 'F:/人口数据/origin_change/ppp_{year}_1km_Aggregated.tif'  # 调整后的人口数据文件模板

for year in years:
    # 格式化文件路径
    tif_file_1 = tif_file_1_template.format(year=year)
    tif_file_2 = tif_file_2_template.format(year=year)
    output_file = output_file_template.format(year=year)

    # 确保输出目录存在
    output_folder = os.path.dirname(output_file)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 打开热指数数据文件以获取其参考信息
    with rasterio.open(tif_file_1) as src1:
        dst_crs = src1.crs
        dst_transform = src1.transform
        dst_width = src1.width
        dst_height = src1.height
        dst_bounds = src1.bounds
        dst_profile = src1.profile

        # 打开人口数据文件
        with rasterio.open(tif_file_2) as src2:
            # 设置输出配置文件，使用与热指数数据相同的配置
            profile = dst_profile
            profile.update({
                'dtype': src2.dtypes[0],  # 保持原始数据类型
                'compress': 'lzw'  # 可以选择使用压缩以减少文件大小
            })

            # 创建输出文件并进行重新投影和裁剪
            with rasterio.open(output_file, 'w', **profile) as dst:
                for i in range(1, src2.count + 1):
                    reproject(
                        source=rasterio.band(src2, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src2.transform,
                        src_crs=src2.crs,
                        dst_transform=dst_transform,
                        dst_crs=dst_crs,
                        dst_resolution=(dst_transform[0], dst_transform[4]),  # 使用目标文件的分辨率
                        resampling=Resampling.nearest  # 使用最近邻插值法
                    )

        print(f"人口数据文件已调整并保存到: {output_file}")

In [ ]:
#将人均cdd数据乘以人口数据得到总的cdd数据  多个年份

In [ ]:
import rasterio
import numpy as np
import os

# 定义年份范围
years = range(2005, 2021)

# 定义输入和输出文件模板
temperature_file_template = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km/re_ssp126_{year}.tif'
population_file_template = 'F:/人口数据/origin_change/ppp_{year}_1km_Aggregated.tif'
output_file_template = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km_popSUM/re_ssp126_{year}.tif'

# 遍历每一个年份，处理对应的栅格文件
for year in years:
    # 格式化文件路径
    temperature_file = temperature_file_template.format(year=year)
    population_file = population_file_template.format(year=year)
    output_file = output_file_template.format(year=year)

    # 确保输出目录存在
    output_folder = os.path.dirname(output_file)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 打开气温栅格数据文件
    with rasterio.open(temperature_file) as temp_src:
        temp_data = temp_src.read(1)  # 读取第一个波段的数据
        temp_profile = temp_src.profile  # 获取气温栅格数据的元数据
        temp_nodata = temp_src.nodata  # 获取气温栅格数据的NoData值

    # 打开人口栅格数据文件
    with rasterio.open(population_file) as pop_src:
        pop_data = pop_src.read(1)  # 读取第一个波段的数据
        pop_nodata = pop_src.nodata  # 获取人口栅格数据的NoData值

    # 检查两个栅格的形状是否相同
    if temp_data.shape != pop_data.shape:
        raise ValueError(f"Temperature and population raster shapes do not match for year {year}!")

    # 计算气温和人口数据的乘积
    product_data = np.where(
        (temp_data == temp_nodata) | (pop_data == pop_nodata),  # 如果任何一个栅格中的值是NoData
        temp_nodata,  # 保持NoData
        temp_data * pop_data  # 否则，计算乘积
    )

    # 更新元数据以确保保存时格式正确
    temp_profile.update(dtype=rasterio.float32)

    # 保存结果到新的栅格文件
    with rasterio.open(output_file, 'w', **temp_profile) as dst:
        dst.write(product_data.astype(rasterio.float32), 1)

    print(f"Year {year} processed. Result saved to: {output_file}")

print("所有年份的计算已完成。")

In [ ]:
import rasterio
import numpy as np
import os

# 定义年份范围
years = range(2019, 2020)

# 定义输入和输出文件模板
temperature_file_template = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km/re_ssp126_{year}.tif'
population_file_template = 'F:/人口数据/lanscan_origin/landscan-global-{year}-assets/landscan-global-{year}_360.tif'
output_file_template = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/0ssp126_rasters_1km_popSUM_landscan/re_ssp126_{year}.tif'

# 遍历每一个年份，处理对应的栅格文件
for year in years:
    # 格式化文件路径
    temperature_file = temperature_file_template.format(year=year)
    population_file = population_file_template.format(year=year)
    output_file = output_file_template.format(year=year)

    # 确保输出目录存在
    output_folder = os.path.dirname(output_file)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 打开气温栅格数据文件
    with rasterio.open(temperature_file) as temp_src:
        temp_data = temp_src.read(1)  # 读取第一个波段的数据
        temp_profile = temp_src.profile  # 获取气温栅格数据的元数据
        temp_nodata = temp_src.nodata  # 获取气温栅格数据的NoData值

    # 打开人口栅格数据文件
    with rasterio.open(population_file) as pop_src:
        pop_data = pop_src.read(1)  # 读取第一个波段的数据
        pop_nodata = pop_src.nodata  # 获取人口栅格数据的NoData值

    # 检查两个栅格的形状是否相同
    if temp_data.shape != pop_data.shape:
        raise ValueError(f"Temperature and population raster shapes do not match for year {year}!")

    # 计算气温和人口数据的乘积
    product_data = np.where(
        (temp_data == temp_nodata) | (pop_data == pop_nodata),  # 如果任何一个栅格中的值是NoData
        temp_nodata,  # 保持NoData
        temp_data * pop_data  # 否则，计算乘积
    )

    # 更新元数据以确保保存时格式正确
    temp_profile.update(dtype=rasterio.float32)

    # 保存结果到新的栅格文件
    with rasterio.open(output_file, 'w', **temp_profile) as dst:
        dst.write(product_data.astype(rasterio.float32), 1)

    print(f"Year {year} processed. Result saved to: {output_file}")

print("所有年份的计算已完成。")

In [ ]:
#将人均cdd数据乘以人口数据得到总的cdd数据   单个年份

In [ ]:
import rasterio
import numpy as np
import os

# 定义输入和输出文件路径
temperature_file = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km/re_ssp126_2019.tif'  # 气温栅格数据
population_file = 'F:/人口数据/origin_change/ppp_2010_1km_Aggregated.tif'  # 人口栅格数据
output_file = 'M:/2015-2020data/jisuan18.3_0.1_pop_year_0.125_SUM_change/ssp126_rasters_1km_popSUM/re_ssp126_19clim_10pop.tif'  # 输出结果文件

# 确保输出目录存在
output_folder = os.path.dirname(output_file)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 打开气温栅格数据文件
with rasterio.open(temperature_file) as temp_src:
    temp_data = temp_src.read(1)  # 读取第一个波段的数据
    temp_profile = temp_src.profile  # 获取气温栅格数据的元数据
    temp_nodata = temp_src.nodata  # 获取气温栅格数据的NoData值

# 打开人口栅格数据文件
with rasterio.open(population_file) as pop_src:
    pop_data = pop_src.read(1)  # 读取第一个波段的数据
    pop_nodata = pop_src.nodata  # 获取人口栅格数据的NoData值

# 检查两个栅格的形状是否相同
if temp_data.shape != pop_data.shape:
    raise ValueError("Temperature and population raster shapes do not match!")

# 计算气温和人口数据的乘积
product_data = np.where(
    (temp_data == temp_nodata) | (pop_data == pop_nodata),  # 如果任何一个栅格中的值是NoData
    temp_nodata,  # 保持NoData
    temp_data * pop_data  # 否则，计算乘积
)

# 更新元数据以确保保存时格式正确
temp_profile.update(dtype=rasterio.float32)

# 保存结果到新的栅格文件
with rasterio.open(output_file, 'w', **temp_profile) as dst:
    dst.write(product_data.astype(rasterio.float32), 1)

print(f"计算完成，结果已保存到：{output_file}")